In [3]:
import pandas as pd
import pickle as pkl
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style = "darkgrid")

In [ ]:
'''
!pip install pandas
!pip install seaborn
!pip install fsspec
!pip install s3fs
!pip install boto
'''

### pkmgo feature

- appearedHour (c)
- appearedTimeOfDay (c)
- appearedDayOfWeek (c)
- appearedMonth (c)
- terrainType (c)
- closeToWater (01)
- weather (c)

- temperature (n)
- windSpeed (n)
- windBearing (n)
- pressure (n)
- sunriseMinutesMidnight (n)
- population density (n)
- urban-rural (c)
- gymDistanceKm (n)
- pokestopDistanceKm (n)

- coocc_xxx (b)

### description
- pokemonId - the identifier of a pokemon, should be deleted to not affect predictions. (numeric; ranges between 1 and 151)
- latitude, longitude - coordinates of a sighting (numeric)
- appearedLocalTime - exact time of a sighting in format yyyy-mm-dd'T'hh-mm-ss.ms'Z' (nominal)
- cellId 90-5850m - geographic position projected on a S2 Cell, with cell sizes ranging from 90 to 5850m (numeric)
- appearedTimeOfDay - time of the day of a sighting (night, evening, afternoon, morning)
- appearedHour/appearedMinute - local hour/minute of a sighting (numeric)
- appearedDayOfWeek - week day of a sighting (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday)
- appearedDay/appearedMonth/appearedYear - day/month/year of a sighting (numeric)
- terrainType - terrain where pokemon appeared described with help of GLCF Modis Land Cover (numeric)
- closeToWater - did pokemon appear close (100m or less) to water (Boolean, same source as above)
- city - the city of a sighting (nominal)
- continent (not always parsed right) - the continent of a sighting (nominal)
- weather - weather type during a sighting (Foggy Clear, PartlyCloudy, MostlyCloudy, Overcast, Rain, BreezyandOvercast, LightRain, Drizzle, BreezyandPartlyCloudy, HeavyRain, BreezyandMostlyCloudy, Breezy, Windy, WindyandFoggy, Humid, Dry, WindyandPartlyCloudy, DryandMostlyCloudy, DryandPartlyCloudy, DrizzleandBreezy, LightRainandBreezy, HumidandPartlyCloudy, HumidandOvercast, RainandWindy) // Source for all weather features
- temperature - temperature in celsius at the location of a sighting (numeric)
- windSpeed - speed of the wind in km/h at the location of a sighting (numeric)
- windBearing - wind direction (numeric)
- pressure - atmospheric pressure in bar at the location of a sighting (numeric)
- weatherIcon - a compact representation of the weather at the location of a sighting (fog, clear-night, partly-cloudy-night, partly-cloudy-day, cloudy, clear-day, rain, wind)
- sunriseMinutesMidnight-sunsetMinutesBefore - time of appearance relatively to sunrise/sunset Source
- population density - what is the population density per square km of a sighting (numeric, Source)
- urban-rural - how urban is location where pokemon appeared (Boolean, built on Population density, <200 for rural, >=200 and <400 for midUrban, >=400 and <800 for subUrban, >800 for urban)
- gymDistanceKm, pokestopDistanceKm - how far is the nearest gym/pokestop in km from a sighting? (numeric, extracted from this dataset)
- gymIn100m-pokestopIn5000m - is there a gym/pokestop in 100/200/etc meters? (Boolean)
- cooc 1-cooc 151 - co-occurrence with any other pokemon (pokemon ids range between 1 and 151) within 100m distance and within the last 24 hours (Boolean)
- class - says which pokemonId it is, to be predicted.

### pkmdex feature
- Stamina (n)
- Attack (n)
- Defense (n)

- type x19 (b) 

- MaxHP (n)
- Capture_rate (n/c)
- Escape_rate (n/c)
- Weight (n)
- Height (n)
- Legendary (b)
- MaxCP (n)

In [5]:
pkmdex = pd.read_csv('data/pkmdex.csv')[:151]
types = set(pkmdex.Primary).union(set(pkmdex.Secondary))
types.remove('None')
for t in types:
    pkmdex[t] = False
    for i, r in pkmdex.iterrows():
        pkmdex.at[i, t] = (t == r.Primary) or ( t == r.Secondary)

In [6]:
pkmgo = pd.read_csv('data/pkmgo.csv')

c:\Users\martin.ranieri\Downloads\spawn-em-all-manzi-2022\manzi\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#"', '".join(df.columns)
_ = [
'Name', 'Pokedex', 'pokemonId', 'class', 'Generation',

'Stamina', 'Attack', 'Defense', 'MaxHP', 'MaxCP',

'Primary', 'Secondary',

'Capture_rate', 'Escape_rate', 'Legendary',

'Weight', 'Height',

# stamp
'latitude', 'longitude', 'appearedLocalTime', '_id',

# S2 SYSTEM
'cellId_90m', 'cellId_180m', 'cellId_370m', 'cellId_730m', 'cellId_1460m', 'cellId_2920m', 'cellId_5850m',

# time
'appearedTimeOfDay', 'appearedHour', 'appearedMinute', 'appearedDayOfWeek', 'appearedDay', 'appearedMonth', 'appearedYear',

# field
'terrainType', 'closeToWater',

# info (2)
'city', 'continent',

#weather
'weather', 'temperature', 'windSpeed', 'windBearing', 'pressure', 'weatherIcon',

# sun (season)
'sunriseMinutesMidnight', 'sunriseHour', 'sunriseMinute', 'sunriseMinutesSince',
'sunsetMinutesMidnight', 'sunsetHour', 'sunsetMinute', 'sunsetMinutesBefore',

'population_density', # pop
'urban', 'suburban', 'midurban', 'rural',

'gymDistanceKm', # gym
'gymIn100m', 'gymIn250m', 'gymIn500m', 'gymIn1000m', 'gymIn2500m', 'gymIn5000m',

'pokestopDistanceKm', # spot
'pokestopIn100m', 'pokestopIn250m', 'pokestopIn500m', 'pokestopIn1000m', 'pokestopIn2500m', 'pokestopIn5000m'
]

In [7]:
cstat = ['Stamina', 'Attack', 'Defense', 'MaxHP', 'MaxCP', 'Capture_rate', 'Escape_rate', 'Weight', 'Height', ]
czone = ['temperature', 'windSpeed', 'pressure', 'pokestopDistanceKm', 'gymDistanceKm', 'population_density',]
ccooc = [f'{t}_x' for t in types] # co-occurrence magnitude
ctype = [f'{t}_y' for t in types] # dex type

In [8]:
def mg(row):
    cooc_table = row[tmpl]
    cooc_table.rename(index=tmpd, inplace=True)
    return tmpt[cooc_table].sum()

In [9]:
tmpl = [f'cooc_{i}' for i in range(1,152)]
tmpd = { f'cooc_{i}':i for i in range(1,152)}
tmpt = pkmdex.set_index('Pokedex')[types]
tt = list(types)

In [10]:
# 9.5 minutes
if False:
    pkmgo[tt] = pkmgo.apply(lambda row: mg(row), axis=1)
    pkl.dump(pkmgo, open('data/pkmgo.pkl', 'wb'))
else:
    pkmgo = pkl.load(open('data/pkmgo.pkl', 'rb'))

In [11]:
df = pd.merge(pkmgo, pkmdex, how='left', left_on='pokemonId', right_on= 'Pokedex')

In [12]:
# cleaning stuff
df = df[df.population_density != 0]
df = df[df.pokestopDistanceKm != '?']

# categorizing stuff
df['log_pd'] = df.population_density.apply(lambda x: int(math.log(x+1,10))+1)

### bayesian network

In [ ]:
from pgmpy.estimators import ParameterEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import ExhaustiveSearch

In [ ]:
inlist = [
    #'appearedHour',
    'appearedDayOfWeek',
    #'appearedMonth',
    #'terrainType',
    'closeToWater',
    #'weather',
    'weatherIcon',
    #'urban', 'suburban', 'midurban', 'rural',
]

outlist = list(types)

In [ ]:
model = BayesianNetwork([(i, o) for i in inlist for o in outlist])

pe = ParameterEstimator(model, df.astype('string'))
mle = MaximumLikelihoodEstimator(model, df.astype('string'))

model.fit(df.astype('string'), estimator=MaximumLikelihoodEstimator)

In [ ]:
'''
print("\n", pe.state_counts('closeToWater'))  # unconditional
print("\n", pe.state_counts('class'))  # conditional
print(mle.estimate_cpd('closeToWater'))  # unconditional
print(mle.estimate_cpd('class'))  # conditional
#'''

In [ ]:
es = ExhaustiveSearch(df[inlist+outlist]) #, scoring_method='bic')
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

### principal component analysis

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def do_pca():
    pca = PCA(n_components=d)
    pc = pca.fit_transform(x)
    pc_df = pd.DataFrame(data = pc, columns = [f'pc{i+1}' for i in range(d)])
    pch_df = pd.concat([pc_df, df[huef]], axis = 1)
    sns.scatterplot(data=pc_df, x="pc1", y="pc2")
    return pc_df

In [ ]:
features = ['Stamina', 'Attack', 'Defense', 'MaxHP', 'MaxCP',] + list(types)
huef = 'Weight'
d = 2

x = pkmdex.loc[:, features].values
y = pkmdex.loc[:, huef].values
x = StandardScaler().fit_transform(x)
xp = do_pca()

In [ ]:
# 3D stuff
'''%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = pc_df['pc1']
y = pc_df['pc2']
z = pc_df['pc3']

ax.scatter(x, y, z)

plt.show()'''

In [ ]:
features = ['temperature', 'windSpeed', 'pressure', 'population_density']
huef = 'Weight'
d = 2

x = df.loc[:, features].values
y = df.loc[:, huef].values
x = StandardScaler().fit_transform(x)
do_pca()

DB SCAN

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
for e in [.35, .50, .75, ]:
    db = DBSCAN(eps=e, min_samples=10).fit(xp)
    labels = db.labels_
    fig = plt.figure(figsize=(5, 5))
    sns.scatterplot(xp.pc1, xp.pc2, hue=["cluster-{}".format(i) for i in labels])

### correspondence analisys

In [ ]:
import prince

In [ ]:
features = ['appearedTimeOfDay', 'weatherIcon', 'log_pd', 'closeToWater']
interesting_types = [f'{t}_y' for t in ['Fire', 'Water', 'Flying', 'Ice', 'Dragon', ]]
for f in features:
    print(set(df[f]))

In [ ]:
mca = prince.MCA()

X = df[features+interesting_types].astype(str)
r = mca.fit(X) # same as calling ca.fs_r(1)
q = mca.transform(X) # same as calling ca.fs_r_sup(df_new) for *another* test set.

In [ ]:
ax = mca.plot_coordinates(
     X=X,
     ax=None,
     figsize=(15, 15),
     show_row_points=False,
     row_points_size=5,
     show_row_labels=False,
     show_column_points=True,
     column_points_size=35,
     show_column_labels=True,
     legend_n_cols=1
)

### canonical correlation

In [ ]:
from sklearn.cross_decomposition import CCA

In [ ]:
'''from collections import defaultdict
d = defaultdict(lambda: 0)
for i in df.pokestopDistanceKm:
    try:
        float(i)
    except:
        d[i]+=1
d'''

In [ ]:
# Split the data in X and Y
X = df[czone]
Y = df[cstat+interesting_types]

# Instantiate the Canonical Correlation Analysis with 2 components
my_cca = CCA(n_components=2)

# Fit the model
my_cca.fit(X, Y)

In [ ]:
label_list = list(X.columns) + list(Y.columns)
color_list = ['blue']*len(X.columns)+['red']*len(Y.columns)

for i in range(len(label_list)):
    if label_list[i].endswith('_x'):
        color_list[i] = 'darkorange'
    if label_list[i].endswith('_y'):
        color_list[i] = 'green'

In [ ]:
# Obtain the rotation matrices
xrot = my_cca.x_rotations_
yrot = my_cca.y_rotations_

# Put them together in a numpy matrix
xyrot = np.vstack((xrot,yrot))

nvariables = xyrot.shape[0]

plt.figure(figsize=(20, 16))
plt.xlim((-0.75,0.99))
plt.ylim((-0.50,0.99))

# Plot an arrow and a text label for each variable
for i in range(nvariables):
  x = xyrot[i,0]
  y = xyrot[i,1]

  plt.arrow(0,0,x,y)
  plt.text(x,y, label_list[i], color=color_list[i])

plt.show()

#### logistic

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=4)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

logreg = LogisticRegression()
logreg.fit(x_train, y_train)

y_pred = logreg.predict(x_test)
print('log: ', metrics.accuracy_score(y_test, y_pred))

In [ ]:
#pal = dict(True="#6495ED", False="#F08080")

# Show the survival probability as a function of age and sex
g = sns.lmplot(x="gymDistanceKm", y="Dragon_y", data=df, y_jitter=.02, logistic=True, truncate=False)
#g.set(xlim=(0, 80), ylim=(-.05, 1.05))